In [14]:
import numpy as np 
import pandas as pd

In [15]:
df=pd.read_csv("dataset/survey.csv")
df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [16]:
df.drop(['comments'], axis=1, inplace=True)
df.drop(['state'], axis=1, inplace=True)
df.drop(['Timestamp'], axis=1, inplace=True)
df.drop(['Country'], axis=1, inplace=True)

In [17]:
df['self_employed'] = df['self_employed'].fillna(df['self_employed'].mode()[0])
df['work_interfere'] = df['work_interfere'].fillna(df['work_interfere'].mode()[0])

In [18]:
uncleaned_age_df = df[(df['Age'] < 21) | (df['Age'] > 44)]
len(uncleaned_age_df)
uncleaned_age_df['Age'].index
df = df.drop(uncleaned_age_df['Age'].index)

In [19]:
df['Age'].unique()

array([37, 44, 32, 31, 33, 35, 39, 42, 23, 29, 36, 27, 41, 34, 30, 40, 38,
       24, 28, 26, 22, 25, 21, 43], dtype=int64)

In [20]:
for i in df.columns:
  print(i,df[i].unique())

Age [37 44 32 31 33 35 39 42 23 29 36 27 41 34 30 40 38 24 28 26 22 25 21 43]
Gender ['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'Cis Male' 'Woman' 'f' 'Mal' 'Male (CIS)'
 'queer/she/they' 'non-binary' 'Femake' 'woman' 'Make' 'Nah' 'Enby'
 'fluid' 'Genderqueer' 'Female ' 'Androgyne' 'Agender' 'cis-female/femme'
 'Guy (-ish) ^_^' 'male leaning androgynous' 'Man' 'msle' 'Neuter'
 'Female (trans)' 'queer' 'Mail' 'cis male' 'Malr' 'Male ' 'femail'
 'Cis Man' 'ostensibly male, unsure what that really means']
self_employed ['No' 'Yes']
family_history ['No' 'Yes']
treatment ['Yes' 'No']
work_interfere ['Often' 'Rarely' 'Never' 'Sometimes']
no_employees ['6-25' 'More than 1000' '26-100' '100-500' '1-5' '500-1000']
remote_work ['No' 'Yes']
tech_company ['Yes' 'No']
benefits ['Yes' "Don't know" 'No']
care_options ['Not sure' 'No' 'Yes']
wellness_program ['No' "Don't know" 'Yes']
seek_help ['Yes' "Don't know" 'No']
anonymity ['Yes' "Don't know" 'No

In [21]:
def clean_gender(g):
  male = ['male', 'Male','M', 'm', 'Male-ish', 'maile','Cis Male','Mal', 'Male (CIS)','Make','Male ', 'Man','Guy (-ish) ^_^','male leaning androgynous'
        'msle','cis male', 'Cis Man','ostensibly male, unsure what that really means','Malr','Mail','something kinda male?']
  female = ['Female', 'female','Cis Female', 'F','f','Femake', 'woman','Female ','cis-female/femme','Female (cis)','femail','Woman','female']
  # trans = ['Trans-female','queer/she/they','non-binary','All','fluid', 'Genderqueer','Androgyne', 'Agender',
  #       ,'Trans woman','Neuter', 'Female (trans)','queer','trans']
  if g in male:
    return "male"
  elif g in female:
    return "female"
  else:
    return "female"
  
def clean_leave(x):
  a=['Somewhat difficult','Very difficult']
  b=['Somewhat easy','Very easy']
  if(x in a):
    return "Difficult"
  elif x in b:
    return "easy"
  else:
    return x

def clean_employee(x):
  a=['1-5','6-25']
  b=['100-500','500-1000']
  if(x in a):
    return "1-25"
  elif(x in b):
    return "100-1000"
  else:
    return x
    
def ordinal_encoding(n,k=[]):
  for i in range(len(k)):
    if(n==k[i]):
      return i

def neumeric_yes_no(n):

  if n.lower()=="yes":
    return 0
  else:
    return 1

In [22]:
df["Gender"]=df["Gender"].apply(lambda x:clean_gender(x))
df["Age"]=df["Age"].apply(lambda x:x/100)

l=["phys_health_interview","mental_vs_physical","mental_health_interview","supervisor","obs_consequence","phys_health_consequence","self_employed","tech_company"]
df.drop(l,axis=1,inplace=True)

details=[]
for i in df.columns:
    dict={}
    if i=="leave":
        df[i]=df[i].apply(lambda x:clean_leave(x))
    if i=="wellness_program":
        df[i]=df[i].apply(lambda x:"Yes" if(x=="Don't know") else x)
    if i=="seek_help" or i=="anonymity":
        df[i]=df[i].apply(lambda x:"No" if(x=="Don't know") else x)
    if i=="work_interfere":
        df[i]=df[i].apply(lambda x:"Never" if(x=="Sometimes" or x=="Never") else "Often")
    if i=="no_employees":
        df[i]=df[i].apply(lambda x:clean_employee(x))
        
    dict[i]=df[i].unique()
    details.append(dict)
    
    if i!="Age":
        df[i]=df[i].apply(lambda x:ordinal_encoding(x,df[i].unique()))
    

In [23]:
details

[{'Age': array([0.37, 0.44, 0.32, 0.31, 0.33, 0.35, 0.39, 0.42, 0.23, 0.29, 0.36,
         0.27, 0.41, 0.34, 0.3 , 0.4 , 0.38, 0.24, 0.28, 0.26, 0.22, 0.25,
         0.21, 0.43])},
 {'Gender': array(['female', 'male'], dtype=object)},
 {'family_history': array(['No', 'Yes'], dtype=object)},
 {'treatment': array(['Yes', 'No'], dtype=object)},
 {'work_interfere': array(['Often', 'Never'], dtype=object)},
 {'no_employees': array(['1-25', 'More than 1000', '26-100', '100-1000'], dtype=object)},
 {'remote_work': array(['No', 'Yes'], dtype=object)},
 {'benefits': array(['Yes', "Don't know", 'No'], dtype=object)},
 {'care_options': array(['Not sure', 'No', 'Yes'], dtype=object)},
 {'wellness_program': array(['No', 'Yes'], dtype=object)},
 {'seek_help': array(['Yes', 'No'], dtype=object)},
 {'anonymity': array(['Yes', 'No'], dtype=object)},
 {'leave': array(['easy', "Don't know", 'Difficult'], dtype=object)},
 {'mental_health_consequence': array(['No', 'Maybe', 'Yes'], dtype=object)},
 {'cowor

In [24]:
df["label"]=df.treatment
df.label=df.label+df.family_history*df.work_interfere
df.label=df.label.apply(lambda x:1 if x>0 else 0)
df.label.sum()

787

In [26]:
for i in df.columns:
    if i not in l:
        print(df[i].value_counts())

0.29    85
0.32    82
0.26    75
0.27    71
0.33    70
0.28    68
0.31    67
0.34    65
0.30    63
0.25    61
0.35    55
0.23    51
0.24    46
0.37    43
0.38    39
0.36    37
0.39    33
0.40    33
0.43    28
0.22    21
0.41    21
0.42    20
0.21    16
0.44    11
Name: Age, dtype: int64
1    911
0    250
Name: Gender, dtype: int64
0    697
1    464
Name: family_history, dtype: int64
0    582
1    579
Name: treatment, dtype: int64
1    874
0    287
Name: work_interfere, dtype: int64
0    417
2    267
1    255
3    222
Name: no_employees, dtype: int64
0    829
1    332
Name: remote_work, dtype: int64
0    434
1    374
2    353
Name: benefits, dtype: int64
1    470
2    402
0    289
Name: care_options, dtype: int64
0    786
1    375
Name: wellness_program, dtype: int64
1    943
0    218
Name: seek_help, dtype: int64
1    814
0    347
Name: anonymity, dtype: int64
1    516
0    439
2    206
Name: leave, dtype: int64
0    450
1    450
2    261
Name: mental_health_consequence, dtype: int64
0

In [27]:
for i in df.columns:
  print(i,df[i].unique())

Age [0.37 0.44 0.32 0.31 0.33 0.35 0.39 0.42 0.23 0.29 0.36 0.27 0.41 0.34
 0.3  0.4  0.38 0.24 0.28 0.26 0.22 0.25 0.21 0.43]
Gender [0 1]
family_history [0 1]
treatment [0 1]
work_interfere [0 1]
no_employees [0 1 2 3]
remote_work [0 1]
benefits [0 1 2]
care_options [0 1 2]
wellness_program [0 1]
seek_help [0 1]
anonymity [0 1]
leave [0 1 2]
mental_health_consequence [0 1 2]
coworkers [0 1 2]
label [0 1]


In [28]:
y=df["label"]
x=df.drop(["treatment","label","work_interfere"],axis=1)

In [29]:
x

,Age,Gender,family_history,no_employees,remote_work,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,coworkers
0,0.37,0,0,0,0,0,0,0,0,0,0,0,0
1,0.44,1,0,1,0,1,1,1,1,1,1,1,1
2,0.32,1,0,0,0,2,1,0,1,1,2,0,2
3,0.31,1,1,2,0,2,2,0,1,1,2,2,0
4,0.31,1,0,3,1,0,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,0.36,1,1,1,0,1,1,1,0,1,0,1,0
1254,0.26,1,0,2,0,2,1,0,1,1,0,0,0
1255,0.32,1,1,2,1,0,2,0,1,0,2,0,0
1256,0.34,1,1,1,0,0,2,0,1,1,2,2,1


In [30]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [31]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [48]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(32,activation="relu"),
    tf.keras.layers.Dense(1,activation="relu")
])
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),loss=tf.losses.BinaryCrossentropy(),metrics=["accuracy"])

In [49]:
# lr_scheduler=tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-4*10**(x/20))
history=model.fit(x_train,y_train,epochs=100,verbose=1)#,callbacks=lr_scheduler)
model.evaluate(x_train,y_train),model.evaluate(x_test,y_test)

Epoch 1/100
28/28 [==============================] - 1s 3ms/step - loss: 3.0632 - accuracy: 0.5954
Epoch 2/100
28/28 [==============================] - 0s 2ms/step - loss: 0.8831 - accuracy: 0.5839
Epoch 3/100
28/28 [==============================] - 0s 2ms/step - loss: 0.6719 - accuracy: 0.6287
Epoch 4/100
28/28 [==============================] - 0s 3ms/step - loss: 0.6183 - accuracy: 0.6770
Epoch 5/100
28/28 [==============================] - 0s 2ms/step - loss: 0.6468 - accuracy: 0.6793
Epoch 6/100
28/28 [==============================] - 0s 2ms/step - loss: 0.6655 - accuracy: 0.6425
Epoch 7/100
28/28 [==============================] - 0s 2ms/step - loss: 0.6383 - accuracy: 0.6483
Epoch 8/100
28/28 [==============================] - 0s 3ms/step - loss: 0.9395 - accuracy: 0.5184
Epoch 9/100
28/28 [==============================] - 0s 2ms/step - loss: 0.7081 - accuracy: 0.6690
Epoch 10/100
28/28 [==============================] - 0s 3ms/step - loss: 0.6730 - accuracy: 0.6931
Epoch 11/

([0.45999428629875183, 0.7908046245574951],
 [1.6474093198776245, 0.6666666865348816])

In [46]:
model.save("model2.h5")

In [ ]:
# lrs=1e-4*(10**(tf.range(100)/20))
# plt.figure(figsize=(7,4))
# plt.semilogx(lrs,history.history["loss"])
# plt.xlabel("learning_rate")
# plt.ylabel("Loss")

In [50]:
model.predict(x_test)

10/10 [==============================] - 0s 2ms/step


array([[0.76023793],
       [0.7437158 ],
       [0.43675917],
       [0.4839277 ],
       [0.635019  ],
       [0.706245  ],
       [0.9040117 ],
       [0.43083507],
       [0.6823243 ],
       [0.7224255 ],
       [0.463435  ],
       [0.85865176],
       [0.7204398 ],
       [0.6987008 ],
       [0.3129058 ],
       [0.5253897 ],
       [0.733895  ],
       [1.0919197 ],
       [0.6889165 ],
       [0.29898828],
       [0.8253907 ],
       [1.2445488 ],
       [0.76292455],
       [0.15669423],
       [0.7430973 ],
       [1.0198705 ],
       [1.3767794 ],
       [1.6276394 ],
       [0.6561462 ],
       [0.81683755],
       [0.97123957],
       [0.8564555 ],
       [0.38853884],
       [0.59287643],
       [0.79036987],
       [0.6134268 ],
       [0.5298406 ],
       [0.75811136],
       [0.14803118],
       [0.7211744 ],
       [0.        ],
       [0.6935969 ],
       [0.66514885],
       [0.70298916],
       [0.6041232 ],
       [0.70282185],
       [0.703981  ],
       [0.310